# House Prices: Advanced Regression Techniques

Kaggle Link: https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/overview

This problem is all about predicting the price of a house based on various features available. 

In this Notebook we will be handling the following things;

1. Missing values
2. Year Variables
3. Categorical Variables
4. Standarise the values of the variables to the same range

## Feature Engineering

In [46]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import MinMaxScaler

pd.pandas.set_option('display.max_columns', None)

In [3]:
data = pd.read_csv('data/train.csv')
data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


**NOTE: Usually we perform Feature Engineering only on training data post dividing the dataset into training and testing sets. The Machine Learning model is trained on the training set and tested for accuracy(how well the model performs) on testing data. So, in order to avoid data leakage we have to perform Feature engineering only after dividing the data on training set. Here I am not fitting any model as part of my work, I am only interseted in EDA,FE and Feature Selection of data. Hence am not splitting the data in this use case.**

### Missing Values

In [5]:
# Categorical Features
categoricalFsM = [F for F in data.columns if data[F].isnull().sum()>=1 and data[F].dtypes=='O']

for F in categoricalFsM:
    print("\"{}\": {}% missing values".format(F,np.round(data[F].isnull().mean(),2)))

"Alley": 0.94% missing values
"MasVnrType": 0.01% missing values
"BsmtQual": 0.03% missing values
"BsmtCond": 0.03% missing values
"BsmtExposure": 0.03% missing values
"BsmtFinType1": 0.03% missing values
"BsmtFinType2": 0.03% missing values
"Electrical": 0.0% missing values
"FireplaceQu": 0.47% missing values
"GarageType": 0.06% missing values
"GarageFinish": 0.06% missing values
"GarageQual": 0.06% missing values
"GarageCond": 0.06% missing values
"PoolQC": 1.0% missing values
"Fence": 0.81% missing values
"MiscFeature": 0.96% missing values


In [11]:
#Replacing the missing values with a label "missing"
data[categoricalFsM] = data[categoricalFsM].fillna('Missing')
data[categoricalFsM].isnull().sum()

Alley           0
MasVnrType      0
BsmtQual        0
BsmtCond        0
BsmtExposure    0
BsmtFinType1    0
BsmtFinType2    0
Electrical      0
FireplaceQu     0
GarageType      0
GarageFinish    0
GarageQual      0
GarageCond      0
PoolQC          0
Fence           0
MiscFeature     0
dtype: int64

In [12]:
data[categoricalFsM].head()

,Alley,MasVnrType,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Electrical,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PoolQC,Fence,MiscFeature
0,Missing,BrkFace,Gd,TA,No,GLQ,Unf,SBrkr,Missing,Attchd,RFn,TA,TA,Missing,Missing,Missing
1,Missing,None,Gd,TA,Gd,ALQ,Unf,SBrkr,TA,Attchd,RFn,TA,TA,Missing,Missing,Missing
2,Missing,BrkFace,Gd,TA,Mn,GLQ,Unf,SBrkr,TA,Attchd,RFn,TA,TA,Missing,Missing,Missing
3,Missing,None,TA,Gd,No,ALQ,Unf,SBrkr,Gd,Detchd,Unf,TA,TA,Missing,Missing,Missing
4,Missing,BrkFace,Gd,TA,Av,GLQ,Unf,SBrkr,TA,Attchd,RFn,TA,TA,Missing,Missing,Missing


In [16]:
# Numerical Features
numericalFsM = [F for F in data.columns if data[F].isnull().sum()>=1 and data[F].dtypes!='O']
for F in numericalFsM:
    print("\"{}\": {}% missing value".format(F,np.around(data[F].isnull().mean(),2)))

"LotFrontage": 0.18% missing value
"MasVnrArea": 0.01% missing value
"GarageYrBlt": 0.06% missing value


In [19]:
# Replacing the Numerical Missing Values

for F in numericalFsM:
    median = data[F].median() #calculating median value to replace
    data[F+'_nan'] = np.where(data[F].isnull(),1,0) # Creating a new feature to identify if each of its values were missing or not
    data[F].fillna(median,inplace=True)
    
data[numericalFsM].isnull().sum()

LotFrontage    0
MasVnrArea     0
GarageYrBlt    0
dtype: int64

In [20]:
data[numericalFsM].head(50)

,LotFrontage,MasVnrArea,GarageYrBlt
0,65.0,196.0,2003.0
1,80.0,0.0,1976.0
2,68.0,162.0,2001.0
3,60.0,0.0,1998.0
4,84.0,350.0,2000.0
5,85.0,0.0,1993.0
6,75.0,186.0,2004.0
7,69.0,240.0,1973.0
8,51.0,0.0,1931.0
9,50.0,0.0,1939.0


In [25]:
# DateTime variables
# The year values cannot be used directly. So, we convert them into No. of years passed at the time of selling
yearFs = ['YearBuilt','YearRemodAdd','GarageYrBlt']
for F in yearFs:   
    data[F]=data['YrSold']-data[F]

In [26]:
data[yearFs].head()

,YearBuilt,YearRemodAdd,GarageYrBlt
0,2003,2003,2003.0
1,1976,1976,1976.0
2,2001,2002,2001.0
3,1915,1970,1998.0
4,2000,2000,2000.0


### Handling Numerical Variables

**We shall perform log normal transformation since the data is skewed.**

In [24]:
data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,LotFrontage_nan,MasVnrArea_nan,GarageYrBlt_nan
0,1,60,RL,65.0,8450,Pave,Missing,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,5,5,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,Missing,Attchd,5.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,Missing,Missing,Missing,0,2,2008,WD,Normal,208500,0,0,0
1,2,20,RL,80.0,9600,Pave,Missing,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,31,31,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,31.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,Missing,Missing,Missing,0,5,2007,WD,Normal,181500,0,0,0
2,3,60,RL,68.0,11250,Pave,Missing,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,7,6,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,7.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,Missing,Missing,Missing,0,9,2008,WD,Normal,223500,0,0,0
3,4,70,RL,60.0,9550,Pave,Missing,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,91,36,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,8.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,Missing,Missing,Missing,0,2,2006,WD,Abnorml,140000,0,0,0
4,5,60,RL,84.0,14260,Pave,Missing,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,8,8,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,8.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,Missing,Missing,Missing,0,12,2008,WD,Normal,250000,0,0,0


In [28]:
numericalFs = [F for F in data.columns if data[F].dtypes != 'O']  #columns not of Object type
discreteFs = [F for F in numericalFs if len(data[F].unique())<25 and F not in yearFs+['Id']]
continuousFs = [F for F in numericalFs if F not in discreteFs+ yearFs+ ['Id']]

for F in continuousFs:
    data[F] = np.log(data[F]+0.1)

In [30]:
data[continuousFs].head()

,LotFrontage,LotArea,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,ScreenPorch,SalePrice
0,4.175925,9.041934,5.278625,6.559757,-2.302585,5.011302,6.752387,6.752387,6.750048,7.444307,6.306458,-2.302585,4.112512,-2.302585,-2.302585,12.247695
1,4.383276,9.169529,-2.302585,6.885612,-2.302585,5.649326,7.140532,7.140532,-2.302585,7.140532,6.131444,5.697429,-2.302585,-2.302585,-2.302585,12.109011
2,4.220977,9.328132,5.088213,6.186414,-2.302585,6.073275,6.824482,6.824482,6.764000,7.487790,6.410339,-2.302585,3.740048,-2.302585,-2.302585,12.317167
3,4.096010,9.164307,-2.302585,5.375741,-2.302585,6.291754,6.628174,6.868078,6.628174,7.448392,6.464744,-2.302585,3.558201,5.606170,-2.302585,11.849398
4,4.432007,9.565221,5.858219,6.484788,-2.302585,6.194609,7.043247,7.043247,6.959493,7.695349,6.728748,5.258016,4.432007,-2.302585,-2.302585,12.429217


### Handling Rare Categorical Features

**We shall replace values of categorical variables that are present less than 1% of the observations with a new label.**

In [35]:
categoricalFs = [F for F in data.columns if data[F].dtype=='O']
print(len(categoricalFs),"Categorical Features")
categoricalFs

43 Categorical Features


['MSZoning',
 'Street',
 'Alley',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'PoolQC',
 'Fence',
 'MiscFeature',
 'SaleType',
 'SaleCondition']

In [40]:
for F in categoricalFs:
    temp = data.groupby(F)['SalePrice'].count()/len(data)
    temp_df = list(temp[temp<=0.01].index)
    #print(temp_df)
    data[F] = np.where(data[F].isin(temp_df),'Rare_var',data[F])


In [43]:
data[categoricalFs].head(100)

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Heating,HeatingQC,CentralAir,Electrical,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
0,RL,Pave,Missing,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,BrkFace,Gd,TA,PConc,Gd,TA,No,GLQ,Unf,GasA,Ex,Y,SBrkr,Gd,Typ,Missing,Attchd,RFn,TA,TA,Y,Missing,Missing,Missing,WD,Normal
1,RL,Pave,Missing,Reg,Lvl,AllPub,FR2,Gtl,Rare_var,Feedr,Norm,1Fam,1Story,Gable,CompShg,MetalSd,MetalSd,None,TA,TA,CBlock,Gd,TA,Gd,ALQ,Unf,GasA,Ex,Y,SBrkr,TA,Typ,TA,Attchd,RFn,TA,TA,Y,Missing,Missing,Missing,WD,Normal
2,RL,Pave,Missing,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,BrkFace,Gd,TA,PConc,Gd,TA,Mn,GLQ,Unf,GasA,Ex,Y,SBrkr,Gd,Typ,TA,Attchd,RFn,TA,TA,Y,Missing,Missing,Missing,WD,Normal
3,RL,Pave,Missing,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,Gable,CompShg,Wd Sdng,Wd Shng,None,TA,TA,BrkTil,TA,Gd,No,ALQ,Unf,GasA,Gd,Y,SBrkr,Gd,Typ,Gd,Detchd,Unf,TA,TA,Y,Missing,Missing,Missing,WD,Abnorml
4,RL,Pave,Missing,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,BrkFace,Gd,TA,PConc,Gd,TA,Av,GLQ,Unf,GasA,Ex,Y,SBrkr,Gd,Typ,TA,Attchd,RFn,TA,TA,Y,Missing,Missing,Missing,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,RL,Pave,Missing,IR2,Lvl,AllPub,Corner,Gtl,Gilbert,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,BrkFace,Ex,Gd,PConc,Gd,Gd,No,ALQ,Unf,GasA,Gd,Y,SBrkr,TA,Typ,TA,BuiltIn,Fin,TA,TA,Y,Missing,Missing,Shed,WD,Normal
96,RL,Pave,Missing,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,1Story,Gable,CompShg,VinylSd,VinylSd,BrkFace,Gd,TA,PConc,Gd,TA,Av,ALQ,Unf,GasA,Ex,Y,SBrkr,Gd,Typ,Missing,Attchd,RFn,TA,TA,Y,Missing,Missing,Missing,WD,Normal
97,RL,Pave,Missing,Reg,HLS,AllPub,Inside,Gtl,Edwards,Norm,Norm,1Fam,1Story,Hip,CompShg,HdBoard,HdBoard,BrkFace,TA,TA,CBlock,TA,TA,No,Rec,Unf,GasA,TA,Y,FuseF,TA,Typ,Missing,Attchd,Fin,TA,TA,P,Missing,Missing,Missing,WD,Normal
98,RL,Pave,Missing,Reg,Lvl,AllPub,Corner,Gtl,Edwards,Norm,Norm,1Fam,1Story,Gable,CompShg,Wd Sdng,Wd Sdng,None,TA,TA,BrkTil,TA,TA,No,ALQ,Unf,GasA,Fa,N,SBrkr,TA,Typ,Missing,Basment,Unf,Fa,TA,Y,Missing,Missing,Shed,COD,Abnorml


**We shall replace thae category values with integers as it becomes easier to handle numerical values.**<br>
**For eg., If a variable has values 'A','B' and 'C', then we shall replace them with '1','2' and '3' in ascending order based on the average salePrice. Other popular ways can be to Label or OneHot Encode the values.**

In [44]:
for F in categoricalFs:
    labels = data.groupby([F])['SalePrice'].mean().sort_values().index
    labels = {k:i for i,k in enumerate(labels,0)}
    data[F]=data[F].map(labels)

In [45]:
data[categoricalFs].head(30)

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Heating,HeatingQC,CentralAir,Electrical,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
0,3,1,2,0,1,1,0,0,14,2,1,3,5,0,0,10,10,2,2,3,4,3,3,1,6,5,2,4,1,3,2,4,1,4,2,2,3,2,0,4,2,2,3
1,3,1,2,0,1,1,2,0,11,1,1,3,3,0,0,4,3,1,1,3,2,3,3,4,4,5,2,4,1,3,1,4,3,4,2,2,3,2,0,4,2,2,3
2,3,1,2,1,1,1,0,0,14,2,1,3,5,0,0,10,10,2,2,3,4,3,3,2,6,5,2,4,1,3,2,4,3,4,2,2,3,2,0,4,2,2,3
3,3,1,2,1,1,1,1,0,16,2,1,3,5,0,0,2,4,1,1,3,1,2,4,1,4,5,2,3,1,3,2,4,4,2,1,2,3,2,0,4,2,2,0
4,3,1,2,1,1,1,2,0,22,2,1,3,5,0,0,10,10,2,2,3,4,3,3,3,6,5,2,4,1,3,2,4,3,4,2,2,3,2,0,4,2,2,3
5,3,1,2,1,1,1,0,0,9,2,1,3,1,0,0,10,10,1,1,3,3,3,3,1,6,5,2,4,1,3,1,4,1,4,1,2,3,2,0,2,1,2,3
6,3,1,2,0,1,1,0,0,18,2,1,3,3,0,0,10,10,4,2,3,4,4,3,3,6,5,2,4,1,3,2,4,4,4,2,2,3,2,0,4,2,2,3
7,3,1,2,1,1,1,1,0,12,5,1,3,5,0,0,6,5,4,1,3,2,3,3,2,4,1,2,4,1,3,1,4,3,4,2,2,3,2,0,4,1,2,3
8,1,1,2,0,1,1,0,0,4,0,1,3,1,0,0,8,4,1,1,3,1,2,3,1,5,5,2,3,1,1,1,3,3,2,1,1,3,2,0,4,2,2,0
9,3,1,2,0,1,1,1,0,3,0,0,0,2,0,0,4,3,1,1,3,1,2,3,1,6,5,2,4,1,3,1,4,3,4,2,3,3,2,0,4,2,2,3


## Feature Scaling

In [47]:
scalingFs = [F for F in data.columns if F not in ['Id','SalePrice']]

scaler = MinMaxScaler()
scaler.fit(data[scalingFs])

MinMaxScaler()

In [49]:
scaledVals = scaler.transform(data[scalingFs])
scaledVals

array([[0.23529412, 0.75      , 0.41770384, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.75      , 0.49457893, ..., 0.        , 0.        ,
        0.        ],
       [0.23529412, 0.75      , 0.43440703, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.29411765, 0.75      , 0.4233556 , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.75      , 0.43440703, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.75      , 0.47068225, ..., 0.        , 0.        ,
        0.        ]])

In [51]:
# Concatenating the ['Id','SalePrice'] columns along with 'scaledvals'(numpy array) to get final DataFrame 
data = pd.concat([data[['Id', 'SalePrice']].reset_index(drop=True),
                    pd.DataFrame((scaledVals), columns=scalingFs)],
                    axis=1)
data.head()

,Id,SalePrice,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,LotFrontage_nan,MasVnrArea_nan,GarageYrBlt_nan
0,1,12.247695,0.235294,0.75,0.417704,0.366337,1.0,1.0,0.000000,0.333333,1.0,0.00,0.0,0.636364,0.4,1.0,0.75,1.0,0.666667,0.500,0.949275,0.883333,0.0,0.0,1.0,1.0,0.50,0.783150,0.666667,1.0,1.00,0.75,0.75,0.25,1.000000,0.810016,0.833333,0.0,0.727112,0.821664,1.0,1.00,1.0,1.0,0.356123,0.911138,0.0,0.577685,0.333333,0.0,0.666667,0.5,0.375,0.333333,0.666667,0.500000,1.0,0.000000,0.2,0.8,0.936364,0.666667,0.50,0.900560,0.666667,1.0,1.0,0.000000,0.745316,0.000000,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.090909,0.50,0.666667,0.75,0.0,0.0,0.0
1,2,12.109011,0.000000,0.75,0.494579,0.391310,1.0,1.0,0.000000,0.333333,1.0,0.50,0.0,0.500000,0.2,1.0,0.75,0.6,0.555556,0.875,0.753623,0.433333,0.0,0.0,0.4,0.3,0.25,0.000000,0.333333,1.0,0.50,0.75,0.75,1.00,0.666667,0.839799,0.833333,0.0,0.790541,0.856885,1.0,1.00,1.0,1.0,0.503026,0.000000,0.0,0.470213,0.000000,0.5,0.666667,0.0,0.375,0.333333,0.333333,0.333333,1.0,0.333333,0.6,0.8,0.690909,0.666667,0.50,0.882252,0.666667,1.0,1.0,0.883380,0.000000,0.000000,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.363636,0.25,0.666667,0.75,0.0,0.0,0.0
2,3,12.317167,0.235294,0.75,0.434407,0.422352,1.0,1.0,0.333333,0.333333,1.0,0.00,0.0,0.636364,0.4,1.0,0.75,1.0,0.666667,0.500,0.934783,0.866667,0.0,0.0,1.0,1.0,0.50,0.763480,0.666667,1.0,1.00,0.75,0.75,0.50,1.000000,0.775892,0.833333,0.0,0.832688,0.828206,1.0,1.00,1.0,1.0,0.383409,0.912543,0.0,0.593068,0.333333,0.0,0.666667,0.5,0.375,0.333333,0.666667,0.333333,1.0,0.333333,0.6,0.8,0.918182,0.666667,0.50,0.911426,0.666667,1.0,1.0,0.000000,0.702043,0.000000,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.727273,0.50,0.666667,0.75,0.0,0.0,0.0
3,4,11.849398,0.294118,0.75,0.388076,0.390288,1.0,1.0,0.333333,0.333333,1.0,0.25,0.0,0.727273,0.4,1.0,0.75,1.0,0.666667,0.500,0.311594,0.333333,0.0,0.0,0.2,0.4,0.25,0.000000,0.333333,1.0,0.25,0.50,1.00,0.25,0.666667,0.701797,0.833333,0.0,0.854408,0.810393,1.0,0.75,1.0,1.0,0.399909,0.898872,0.0,0.579130,0.333333,0.0,0.333333,0.0,0.375,0.333333,0.666667,0.416667,1.0,0.333333,0.8,0.4,0.890909,0.333333,0.75,0.917117,0.666667,1.0,1.0,0.000000,0.680915,0.917881,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.090909,0.00,0.666667,0.00,0.0,0.0,0.0
4,5,12.429217,0.235294,0.75,0.512646,0.468754,1.0,1.0,0.333333,0.333333,1.0,0.50,0.0,1.000000,0.4,1.0,0.75,1.0,0.777778,0.500,0.927536,0.833333,0.0,0.0,1.0,1.0,0.50,0.843023,0.666667,1.0,1.00,0.75,0.75,0.75,1.000000,0.803164,0.833333,0.0,0.844750,0.848057,1.0,1.00,1.0,1.0,0.466206,0.932219,0.0,0.666499,0.333333,0.0,0.666667,0.5,0.500,0.333333,0.666667,0.583333,1.0,0.333333,0.6,0.8,0.909091,0.666667,0.75,0.944734,0.666667,1.0,1.0,0.834859,0.782435,0.000000,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.000000,0.50,0.666667,0.75,0.0,0.0,0.0


In [53]:
data.to_csv('data/X_train.csv',index=False)